<a href="https://colab.research.google.com/github/Sushilbokade/Nvidia-Components-Hands-On/blob/main/Agentic%20AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hands-On Exercise!

#  NeMo Agent Toolkit — Hello World (Stable Colab Tutorial)

This notebook runs the [NeMo Agent Toolkit](https://github.com/NVIDIA/NeMo-Agent-Toolkit)'s Hello World example using a ReAct agent, Wikipedia tool, and NVIDIA's hosted LLaMA-3.1 via NIM.

**You'll learn:**
- What a ReAct agent is
- How it uses tools + LLMs to solve queries
- How to run the whole setup inside Colab

 You must have a [NVIDIA API key](https://build.nvidia.com) to run this example.


In [1]:
# ✅ Step 1: Clean any broken or cached directories (safe restart)
import os
os.chdir("/content")
!rm -rf /content/aiqtoolkit


In [2]:
# ✅ Step 2: Clone NeMo Agent Toolkit and move into it
!git clone https://github.com/NVIDIA/NeMo-Agent-Toolkit.git /content/aiqtoolkit
import os
os.chdir("/content")


Cloning into '/content/aiqtoolkit'...
remote: Enumerating objects: 8661, done.
remote: Counting objects: 100% (395/395), done.
remote: Compressing objects: 100% (265/265), done.
remote: Total 8661 (delta 254), reused 134 (delta 128), pack-reused 8266 (from 3)
Receiving objects: 100% (8661/8661), 37.25 MiB | 28.70 MiB/s, done.
Resolving deltas: 100% (4380/4380), done.


In [3]:
# ✅ Step 3: Install `uv` and create virtual environment
!pip install -q uv
!uv venv --seed .venv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.8/18.8 MB 76.1 MB/s eta 0:00:00
Using CPython 3.11.13 interpreter at: /usr/bin/python3
Creating virtual environment with seed packages at: .venv
 + pip==25.2
 + setuptools==80.9.0
 + wheel==0.45.1
Activate with: source .venv/bin/activate


In [4]:
# ✅ Step 4: Activate env and install toolkit + all extras
%%bash
cd /content/aiqtoolkit
source .venv/bin/activate
uv sync --all-groups --all-extras

bash: line 2: .venv/bin/activate: No such file or directory
Using CPython 3.12.11
Creating virtual environment at: .venv
Resolved 521 packages in 1m 28s
   Building aiqtoolkit-test @ file:///content/aiqtoolkit/packages/aiqtoolkit_test
   Building aiq-user-report @ file:///content/aiqtoolkit/examples/object_store/user_report
   Building aiq-agno-personal-finance @ file:///content/aiqtoolkit/examples/frameworks/agno_personal_finance
   Building aiqtoolkit-ragaai @ file:///content/aiqtoolkit/packages/aiqtoolkit_ragaai
   Building aiqtoolkit-agno @ file:///content/aiqtoolkit/packages/aiqtoolkit_agno
   Building aiqtoolkit-crewai @ file:///content/aiqtoolkit/packages/aiqtoolkit_crewai
   Building aiq-alert-triage-agent @ file:///content/aiqtoolkit/examples/advanced_agents/alert_triage_agent
   Building aiq-automated-description-generation @ file:///content/aiqtoolkit/examples/custom_functions/automated_description_generation
   Building aiq-email-phishing-analyzer @ file:///content/aiqtoolk

In [5]:
# ✅ Step 5: Set your NVIDIA API key (required for NIM LLM)
api_key = "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" # 🔒 Replace with real key
with open("/content/aiqtoolkit/env.sh", "w") as f:
    f.write(f"export NVIDIA_API_KEY={api_key}\n")

In [6]:
# ✅ Step 6: Write the agent config (workflow.yaml)
%%bash
cd /content/aiqtoolkit
cat <<EOF > workflow.yaml
functions:
  wikipedia_search:
    _type: wiki_search
    max_results: 2

llms:
  nim_llm:
    _type: nim
    model_name: meta/llama-3.1-70b-instruct
    temperature: 0.0

workflow:
  _type: react_agent
  tool_names: [wikipedia_search]
  llm_name: nim_llm
  verbose: true
  parse_agent_response_max_retries: 3
EOF

### 🧠 What Does the Config Do?
- `functions`: Defines the Wikipedia tool the agent can use
- `llms`: Uses the LLaMA 3.1-70b model hosted by NVIDIA NIM
- `workflow`: Specifies a `react_agent`, which reasons + acts via the tool
- `verbose`: Enables detailed tracing of agent thoughts/actions

In [7]:
# ✅ Step 7: Run the Hello World agent
%%bash
cd /content/aiqtoolkit
source .venv/bin/activate
source env.sh
aiq run --config_file workflow.yaml --input "Write a python code to reverse a string"


Configuration Summary:
--------------------
Workflow Type: react_agent
Number of Functions: 1
Number of LLMs: 1
Number of Embedders: 0
Number of Memory: 0
Number of Object Stores: 0
Number of Retrievers: 0
Number of ITS Strategies: 0
Number of Authentication Providers: 0



2025-08-01 06:55:00,281 - aiq.runtime.loader - WARNING - Loading module 'aiq_profiler_agent.register' from entry point 'aiq_profiler_agent' took a long time (1470.003605 ms). Ensure all imports are inside your registered functions.
2025-08-01 06:55:01,826 - aiq.runtime.loader - WARNING - Loading module 'aiq_automated_description_generation.register' from entry point 'aiq_automated_description_generation' took a long time (1517.140627 ms). Ensure all imports are inside your registered functions.
2025-08-01 06:55:01,983 - aiq.runtime.loader - WARNING - Loading module 'aiq.test.register' from entry point 'aiqtoolkit-test' took a long time (138.629913 ms). Ensure all imports are inside your registered functions.
2025-08-01 06:55:02,265 - aiq.runtime.loader - WARNING - Loading module 'aiq_multi_frameworks.register' from entry point 'aiq_multi_frameworks' took a long time (281.983614 ms). Ensure all imports are inside your registered functions.
2025-08-01 06:55:02,630 - aiq.runtime.loader - 

🎉 If everything is configured properly, the agent will:
1. Think about how to solve the query
2. Use the Wikipedia search tool
3. Call LLaMA 3.1 to formulate a response
4. Print the answer in console
